In [ ]:
# VIX Data Analysis Notebook

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

start_date = '1990-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

def get_data(ticker, start_date, end_date):
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        return df
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

print("Fetching VIX data...")
vix_data = get_data("^VIX", start_date, end_date)

print("Fetching S&P 500 data...")
sp500_data = get_data("^GSPC", start_date, end_date)

print("Fetching VVIX data...")
vvix_data = get_data("^VVIX", start_date, end_date)

print("Fetching SKEW index data...")
skew_data = get_data("^SKEW", start_date, end_date)

if not vix_data.empty:
    vix_data['VIX_20_SMA'] = vix_data['Close'].rolling(window=20).mean()
    vix_data['VIX_50_SMA'] = vix_data['Close'].rolling(window=50).mean()
    print("Calculated VIX SMAs")

def calculate_atr(df, period=10): 
    df = df.copy()
    high = df['High']
    low = df['Low']
    close = df['Close'].shift(1)
    
    tr1 = high - low
    tr2 = abs(high - close)
    tr3 = abs(low - close)
    
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    atr = tr.rolling(window=period).mean()
    
    return atr

if not sp500_data.empty:
    sp500_data['ATR_10'] = calculate_atr(sp500_data, 10)
    print("Calculated 2-week ATR for S&P 500")

result_df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date))

if not vix_data.empty:
    result_df['VIX_Close'] = vix_data['Close']
    result_df['VIX_20_SMA'] = vix_data['VIX_20_SMA']
    result_df['VIX_50_SMA'] = vix_data['VIX_50_SMA']

if not vvix_data.empty:
    result_df['VVIX_Close'] = vvix_data['Close']

if not skew_data.empty:
    result_df['SKEW_Close'] = skew_data['Close']

if not sp500_data.empty:
    result_df['SP500_ATR_10'] = sp500_data['ATR_10']

result_df = result_df[result_df.index.isin(vix_data.index)]

result_df = result_df.round(2)

print("\nData Preview:")
result_df.head()

print("\nData Statistics:")
result_df.describe()

plt.figure(figsize=(12, 6))
plt.plot(result_df.index, result_df['VIX_Close'], label='VIX Close')
plt.plot(result_df.index, result_df['VIX_20_SMA'], label='VIX 20-day SMA')
plt.plot(result_df.index, result_df['VIX_50_SMA'], label='VIX 50-day SMA')
plt.title('VIX and Moving Averages')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

result_df.to_csv('vix_data_analysis.csv')
print("\nData saved to 'vix_data_analysis.csv'")

result_df